In [1]:
from google.colab import drive
import pandas as pd
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.cluster import AgglomerativeClustering
import numpy as np
import pandas as pd
import random
from sklearn.preprocessing import StandardScaler,LabelEncoder

In [2]:
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
train=pd.read_csv('/content/drive/MyDrive/CIC-IDS-2018(15).csv')

<ipython-input-3-cacbf28de427>:1: DtypeWarning: Columns (0,1,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78) have mixed types. Specify dtype option on import or set low_memory=False.
  train=pd.read_csv('/content/drive/MyDrive/CIC-IDS-2018(15).csv')


In [4]:
train.drop(train.loc[train["Label"] == "Label"].index, inplace=True)
train["Protocol"].unique()
train = train.astype({"Protocol": str})
train["Protocol"].unique()
train = pd.get_dummies(train, columns=['Protocol'], drop_first=True)

In [5]:
columns_to_drop = [
    'Dst Port',
    'Timestamp',
    'Fwd PSH Flags',
    'Bwd PSH Flags',
    'Fwd URG Flags',
    'Bwd URG Flags',
    'Flow Byts/s',
    'Flow Pkts/s'
]
train.drop(columns=columns_to_drop, inplace=True)

In [7]:
train.dropna(inplace=True)

In [8]:
train.drop_duplicates(inplace=True)

In [9]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
train['Label'] = encoder.fit_transform(train['Label'])

In [10]:
train = pd.DataFrame(train)
train.replace([np.inf, -np.inf], np.nan, inplace=True)

In [11]:
train.to_csv('/content/drive/MyDrive/cicids_preprocessed_15.csv')

In [12]:
dataset_path = '/content/drive/MyDrive/cicids_preprocessed_15.csv'
df = pd.read_csv(dataset_path)

features = df.drop(["Label"], axis=1)
labels =  df["Label"]

le = LabelEncoder()
target_class = labels.unique()
labels = le.fit_transform( labels )
X=features
y=labels

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=90)

In [13]:
# Standardize features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [14]:
# Convert numpy arrays to PyTorch tensors
X_train_tensor = torch.FloatTensor(X_train)
y_train_tensor = torch.LongTensor(y_train)
X_test_tensor = torch.FloatTensor(X_test)
y_test_tensor = torch.LongTensor(y_test)

In [15]:
class SimpleMLP(nn.Module):
    def __init__(self):
        super(SimpleMLP, self).__init__()
        self.fc1 = nn.Linear(73, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 14)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return torch.log_softmax(x, dim=1)

In [16]:
def perform_clustering(model, stacked_params, n_clusters):
    clustering = AgglomerativeClustering(n_clusters=n_clusters)
    cluster_labels = clustering.fit_predict(stacked_params.detach().numpy())
    return cluster_labels

In [17]:
def select_clients(train_loaders, cluster_labels):
    selected_clusters = set()
    selected_clients = []
    for loader, label in zip(train_loaders, cluster_labels):
        if label not in selected_clusters:
            selected_clusters.add(label)
            selected_clients.append(loader)
    return selected_clients

In [18]:
def federated_train_with_sa_like_clustering(model, train_loaders, test_loader, num_epochs=5, lr=0.001, threshold_w=0.5, initial_p=8, max_p=8, stabilize_rounds=3, sa_prob=0.4):
    global_model = model
    global_optimizer = optim.Adam(global_model.parameters(), lr=lr)
    p = initial_p
    d = 1
    prev_loss = float('inf')
    stabilize_count = 0

    for epoch in range(num_epochs):
        global_model.train()
        total_loss = 0
        num_batches = 0

        local_params_list = []

        for train_loader in train_loaders:
            local_model = SimpleMLP()
            local_model.load_state_dict(global_model.state_dict())
            local_optimizer = optim.Adam(local_model.parameters(), lr=lr)

            for data, target in train_loader:
                local_optimizer.zero_grad()
                output = local_model(data)
                loss = nn.functional.nll_loss(output, target)
                loss.backward()
                local_optimizer.step()
                total_loss += loss.item()
                num_batches += 1

            local_model.load_state_dict(local_model.state_dict())
            local_params = torch.cat([param.data.view(-1) for param in local_model.parameters()])
            local_params_list.append(local_params.unsqueeze(0))
            stacked_params = torch.cat(local_params_list, dim=0)

        avg_loss = total_loss / num_batches
        reduction_ratio = prev_loss / avg_loss if prev_loss > 0 else 0

        print(f"Epoch {epoch + 1}/{num_epochs}: Avg Loss: {avg_loss:.4f}, Reduction Ratio: {reduction_ratio:.4f}, Threshold: {threshold_w}, p: {p}, d: {d}")

        if reduction_ratio > threshold_w:
            # SA-like algorithm: Decide whether to keep p unchanged based on sa_prob
            if random.random() < sa_prob:
                # Keep the number of clusters unchanged
                stabilize_count += 1
            else:
                # Decrease the number of clusters
                stabilize_count = 0
                p = max(p - d, 1)
                d = min(d + 1, max_p - 1)  # Increment d
                print(f"Adjusting clusters DOWN. New number of clusters: {p}, d: {d}")
        else:
            stabilize_count += 1

        if stabilize_count >= stabilize_rounds:
            # Reset d and stabilize count if we've stabilized for enough rounds
            d = 1
            stabilize_count = 0

        prev_loss = avg_loss

        cluster_labels = perform_clustering(global_model, stacked_params, n_clusters=p)

        selected_clients = select_clients(train_loaders, cluster_labels)

        for train_loader in selected_clients:
            local_model = SimpleMLP()
            local_model.load_state_dict(global_model.state_dict())
            local_optimizer = optim.Adam(local_model.parameters(), lr=lr)

            for data, target in train_loader:
                local_optimizer.zero_grad()
                output = local_model(data)
                loss = nn.functional.nll_loss(output, target)
                loss.backward()
                local_optimizer.step()

            global_params = global_model.state_dict()
            local_params = local_model.state_dict()
            for key in global_params.keys():
                global_params[key] += local_params[key] / len(selected_clients)

        global_model.load_state_dict(global_params)

        correct = 0
        total = 0
        with torch.no_grad():
            for images, labels in test_loader:
                outputs = global_model(images)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        accuracy = 100 * correct / total
        print(f"Epoch {epoch + 1}/{num_epochs} completed. Accuracy: {accuracy:.2f}%")

    return global_model

In [19]:
num_clients = 8
train_loaders = [DataLoader(TensorDataset(X_train_tensor, y_train_tensor), batch_size=64, shuffle=True) for _ in range(num_clients)]
test_loader = DataLoader(TensorDataset(X_test_tensor, y_test_tensor), batch_size=64, shuffle=False)

In [20]:
global_model = SimpleMLP()
trained_model = federated_train_with_sa_like_clustering(global_model, train_loaders, test_loader, num_epochs=7, initial_p=num_clients, max_p=num_clients, stabilize_rounds=3)

Epoch 1/7: Avg Loss: 0.0904, Reduction Ratio: inf, Threshold: 0.5, p: 8, d: 1
Adjusting clusters DOWN. New number of clusters: 7, d: 2
Epoch 1/7 completed. Accuracy: 98.20%
Epoch 2/7: Avg Loss: 0.0293, Reduction Ratio: 3.0843, Threshold: 0.5, p: 7, d: 2
Adjusting clusters DOWN. New number of clusters: 5, d: 3
Epoch 2/7 completed. Accuracy: 98.12%
Epoch 3/7: Avg Loss: 0.0381, Reduction Ratio: 0.7696, Threshold: 0.5, p: 5, d: 3
Epoch 3/7 completed. Accuracy: 98.45%
Epoch 4/7: Avg Loss: 0.1968, Reduction Ratio: 0.1936, Threshold: 0.5, p: 5, d: 3
Epoch 4/7 completed. Accuracy: 99.00%
Epoch 5/7: Avg Loss: 1.5148, Reduction Ratio: 0.1299, Threshold: 0.5, p: 5, d: 3
Epoch 5/7 completed. Accuracy: 99.40%
Epoch 6/7: Avg Loss: 15.6997, Reduction Ratio: 0.0965, Threshold: 0.5, p: 5, d: 1
Epoch 6/7 completed. Accuracy: 99.60%
Epoch 7/7: Avg Loss: 197.4713, Reduction Ratio: 0.0795, Threshold: 0.5, p: 5, d: 1
Epoch 7/7 completed. Accuracy: 99.67%
